#### Importing necessary libraries

In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # This avoids SettingWithCopyWarnings
import numpy as np

In [3]:
from pandas_datareader import data as pdr
import fix_yahoo_finance as yf
yf.pdr_override()

#### Initializing Variables


In [4]:
data={} # Dictionary to contain pandas dataframe for all the stocks. This is to avoid creating variable for each stock 
        # to store data
n=7 # Period for ATR
f=3 # f is a factor/ multiplier

### Average True Range is generally used as Renko bar size. However we can also use a custom bar size

In [5]:
def ATR(df,n): #df is the DataFrame, n is the period 7,14 ,etc
    df['H-L']=abs(df['HA_High']-df['HA_Low'])
    df['H-PC']=abs(df['HA_High']-df['HA_Close'].shift(1))
    df['L-PC']=abs(df['HA_Low']-df['HA_Close'].shift(1))
    df['TR']=df[['H-L','H-PC','L-PC']].max(axis=1)
    df['ATR']=np.nan
    df.ix[n-1,'ATR']=df['TR'][:n-1].mean()
    for i in range(n,len(df)):
        df['ATR'][i]=(df['ATR'][i-1]*(n-1)+ df['TR'][i])/n
    return 

### Heiken Ashi is one type of chart pattern from Japan that filters out some noise in an effort to better capture the trend.

In [6]:
def HA(df):
    df['HA_Close']=(df['Open']+ df['High']+ df['Low']+ df['Close'])/4
    df['HA_Open']=(df['Open']+df['Close'])/2   
    
    for i in range(1, len(df)):
        df['HA_Open'][i]=(df['HA_Open'][i-1]+df['HA_Close'][i-1])/2 
    df['HA_High']=df[['HA_Open','HA_Close','High']].max(axis=1)
    df['HA_Low']=df[['HA_Open','HA_Close','Low']].min(axis=1)
    return 

### Supertrend is a trend-following indicator and it is plotted on prices and their placement indicates the current trend. 

In [7]:
def ST(df,f,n): #df is the dataframe, n is the period, f is the factor; f=3, n=7 are commonly used.
    #Calculation of SuperTrend. We are using Heiken_Ashi OHLC instead of regular OHLC data
    df['Upper Basic']=(df['HA_High']+df['HA_Low'])/2+(f*df['ATR'])
    df['Lower Basic']=(df['HA_High']+df['HA_Low'])/2-(f*df['ATR'])
    df['Upper Band']=df['Upper Basic']
    df['Lower Band']=df['Lower Basic']
    for i in range(n,len(df)):
        if df['HA_Close'][i-1]<=df['Upper Band'][i-1]:
            df['Upper Band'][i]=min(df['Upper Basic'][i],df['Upper Band'][i-1])
        else:
            df['Upper Band'][i]=df['Upper Basic'][i]    
    for i in range(n,len(df)):
        if df['HA_Close'][i-1]>=df['Lower Band'][i-1]:
            df['Lower Band'][i]=max(df['Lower Basic'][i],df['Lower Band'][i-1])
        else:
            df['Lower Band'][i]=df['Lower Basic'][i]   
    df['SuperTrend']=np.nan
    for i in df['SuperTrend']:
        if df['HA_Close'][n-1]<=df['Upper Band'][n-1]:
            df['SuperTrend'][n-1]=df['Upper Band'][n-1]
        elif df['HA_Close'][n-1]>df['Upper Band'][i]:
            df['SuperTrend'][n-1]=df['Lower Band'][n-1]
    for i in range(n,len(df)):
        if df['SuperTrend'][i-1]==df['Upper Band'][i-1] and df['HA_Close'][i]<=df['Upper Band'][i]:
            df['SuperTrend'][i]=df['Upper Band'][i]
        elif  df['SuperTrend'][i-1]==df['Upper Band'][i-1] and df['HA_Close'][i]>=df['Upper Band'][i]:
            df['SuperTrend'][i]=df['Lower Band'][i]
        elif df['SuperTrend'][i-1]==df['Lower Band'][i-1] and df['HA_Close'][i]>=df['Lower Band'][i]:
            df['SuperTrend'][i]=df['Lower Band'][i]
        elif df['SuperTrend'][i-1]==df['Lower Band'][i-1] and df['HA_Close'][i]<=df['Lower Band'][i]:
            df['SuperTrend'][i]=df['Upper Band'][i]
    return 

### Set of Stocks

In [8]:
StockList=['INFY', 'ICICIBANK'] 

### Fetching data using Yahoo API,  Setting up Pandas Dataframe in the dictionary 'data' with key as Stock name and Value as DataFrame


In [9]:
start='2016-1-1'
for stock in StockList:
    data[stock]=pdr.get_data_yahoo(stock+'.NS', start)

[*********************100%***********************]  1 of 1 downloaded

#### Sometimes due to rate limiting factor, Yahoo api when called yeilds no data. Below cell helps us out in figuring whether we need to ping the api again. 

In [10]:
for stock in data:
    if data[stock].empty:
        print stock

In [11]:
for stock in data:
    print stock, data[stock].head()

ICICIBANK                   Open        High         Low       Close   Adj Close  \
Date                                                                     
2016-01-01  237.544998  239.636002  234.544998  239.091003  229.999863   
2016-01-04  237.272995  237.591003  231.500000  232.317993  223.484390   
2016-01-05  232.955002  234.091003  228.817993  233.363998  224.490616   
2016-01-06  232.182007  233.408997  226.544998  227.363998  218.718781   
2016-01-07  224.000000  225.182007  221.000000  224.317993  215.788589   

              Volume  
Date                  
2016-01-01   5998097  
2016-01-04   9435792  
2016-01-05   8966978  
2016-01-06  17416181  
2016-01-07  18240713  
INFY                    Open         High          Low        Close    Adj Close  \
Date                                                                          
2016-01-01  1100.000000  1108.949951  1093.199951  1105.250000  1049.958252   
2016-01-04  1099.949951  1102.449951  1076.050049  1078.900024  1024

###  Data Cleaning, HA calculation, etc

In [12]:
for stock in data:
    data[stock].drop(data[stock][data[stock].Volume == 0].index, inplace=True) # Data Cleaning
    HA(data[stock]) # Heiken_Ashi Calculation
    ATR(data[stock],n)

### SuperTrend calculation on top of Heiken_Ashi

In [13]:
for stock in data:
    ST(data[stock],f,n)
    data[stock]=data[stock][['HA_Open','HA_High','HA_Low','HA_Close','ATR','SuperTrend']] # Removing unwanted columns
    data[stock]=data[stock].round(2)

In [14]:
data['INFY'].head() #Since ATR period is 7, we will see atr column filling out from 7th row

,HA_Open,HA_High,HA_Low,HA_Close,ATR,SuperTrend
Date,,,,,,
2016-01-01,1102.62,1108.95,1093.20,1101.85,NaN,NaN
2016-01-04,1102.24,1102.45,1076.05,1089.34,NaN,NaN
2016-01-05,1095.79,1095.79,1062.25,1076.83,NaN,NaN
2016-01-06,1086.31,1086.31,1058.90,1069.39,NaN,NaN
2016-01-07,1077.85,1077.85,1047.90,1056.66,NaN,NaN


In [15]:
data['ICICIBANK'].tail(10)

,HA_Open,HA_High,HA_Low,HA_Close,ATR,SuperTrend
Date,,,,,,
2018-03-15,301.68,305.75,300.40,303.34,10.06,330.10
2018-03-16,302.51,302.51,297.40,299.65,9.47,328.37
2018-03-19,301.08,301.08,291.25,295.97,9.52,324.74
2018-03-20,298.53,298.53,290.25,293.11,9.35,322.43
2018-03-21,295.82,295.82,287.35,291.65,9.22,319.25
2018-03-22,293.73,293.73,280.90,285.82,9.74,316.53
2018-03-23,289.78,289.78,273.55,276.01,10.66,313.66
2018-03-26,282.90,283.15,275.95,279.21,10.17,310.06
2018-03-27,281.05,287.30,281.05,284.27,9.87,310.06


#### Taking data to excel sheet

In [16]:
writer = pd.ExcelWriter('Data.xlsx')
for stock in data:
    data[stock].to_excel(writer,stock)
writer.save()